# Setup

In [1]:
using RigidBodyDynamics

Please note that [RigidBodySim.jl](https://github.com/JuliaRobotics/RigidBodySim.jl) now provides a more capable simulation environment.

# Model definition

We'll just use the double pendulum model, loaded from a URDF:

In [2]:
urdf = joinpath(dirname(pathof(RigidBodyDynamics)), "..", "test", "urdf", "Acrobot.urdf")
mechanism = parse_urdf(Float64, urdf)
remove_fixed_tree_joints!(mechanism)
mechanism

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow
No non-tree joints.

# Controller

Let's write a simple controller that just applies $10 \sin(t)$ at the elbow joint and adds some damping at the shoulder joint:

In [3]:
shoulder, elbow = joints(mechanism)
function simple_control!(torques::AbstractVector, t, state::MechanismState)
    torques[velocity_range(state, shoulder)] .= -1 .* velocity(state, shoulder)
    torques[velocity_range(state, elbow)] .= 10 * sin(t)
end

simple_control! (generic function with 1 method)

# Simulation

Basic simulation can be done using the `simulate` function. We'll first create a `MechanismState` object, and set the initial joint configurations and velocities:

In [4]:
state = MechanismState(mechanism)
zero_velocity!(state)
set_configuration!(state, shoulder, 0.7)
set_configuration!(state, elbow, -0.8);

Now we can simply call `simulate`, which will return a tuple consisting of:
* simulation times (a `Vector` of numbers)
* joint configuration vectors (a `Vector` of `Vector`s)
* joint velocity vectors (a `Vector` of `Vector`s)

In [5]:
final_time = 10.
ts, qs, vs = simulate(state, final_time, simple_control!; Δt = 1e-3);

For access to lower-level functionality, such as different ways of storing or visualizing the data generated during the simulation, it is advised to simply pattern match the basic `simulate` function.

# Visualization

For visualization, we'll use [`RigidBodyTreeInspector`](https://github.com/rdeits/RigidBodyTreeInspector.jl).

(Note: the `#NBSKIP` comments are to skip these cells during `Pkg.test("RigidBodyDynamics")`)

In [6]:
#NBSKIP
using RigidBodyTreeInspector

INFO: Interact.jl: using new nbwidgetsextension protocol


Open the viewer application if it isn't open already:

In [7]:
#NBSKIP
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1));

Load the mechanism's geometry into the visualizer:

In [8]:
#NBSKIP
geometries = visual_elements(mechanism, URDFVisuals(urdf))
vis = Visualizer(mechanism, geometries);

2018-03-21 00:54:37.807 drake-visualizer[40350:6483186] *** WARNING: Method userSpaceScaleFactor in class NSView is deprecated on 10.7 and later. It should not be used in new applications. Use convertRectToBacking: instead. 


And animate:

In [9]:
#NBSKIP
RigidBodyTreeInspector.animate(vis, mechanism, ts, qs; realtimerate = 1.);